In [2]:
%load_ext autoreload
%load_ext watermark
%autoreload 2
%watermark -v -n -m -p numpy,scipy,sklearn,pandas,tensorflow,keras

Sun Feb 24 2019 

CPython 3.6.8
IPython 7.2.0

numpy 1.15.4
scipy 1.2.0
sklearn not installed
pandas 0.23.4
tensorflow 1.12.0
keras 2.2.4

compiler   : GCC 7.3.0
system     : Linux
release    : 4.15.0-43-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit


Using TensorFlow backend.


In [3]:
# TODO visualise and summarise the corpus data

In [20]:
from data.preprocess import PreProcessor

data = PreProcessor(config=PreProcessor.DEFAULT_CONFIG,
                    data_dir='data/raw/r252-corpus-features/org/elasticsearch/action/admin/cluster/allocation/')

vocab = data.metadata['token_vocab']
processed = data.get_tensorise_data()


In [58]:
label_name = np.expand_dims(processed['name_tokens'], 2)

In [ ]:
import numpy as np
from tensorflow.python import keras
from tensorflow.python.keras import layers
from models.cnn_attention import ConvAttention

code_snippet = processed['body_tokens']
label_name = np.expand_dims(processed['name_tokens'], 2)

embedding_dim = 128
vocabulary_size = len(vocab)
max_chunk_length = data.config['max_chunk_length']

# TODO make the input a json file and parse it
batch_size = 1
k1 = 8
k2 = 8
w1 = 24
w2 = 29
w3 = 10
dropout_rate = 0.5

# Optimised hyperparameter are reported in page 5 of the paper

# define layers
main_input = layers.Input(shape=(max_chunk_length,),
                          batch_size=1,
                          dtype='int32', name='main_input')
embedding_layer = layers.Embedding(vocabulary_size,
                                   embedding_dim,
                                   input_length=max_chunk_length)
gru_layer = keras.layers.GRU(k2, return_state=True)
cnn_layer = ConvAttention(k1=k1, k2=k2, w1=w1, w2=w2, w3=w3, dropout_rate=dropout_rate)
optimizer = keras.optimizers.Nadam()  # RMSprop with Nesterov momentum
loss_func = keras.losses.mean_squared_error

# define execution
tokens_embedded = embedding_layer(main_input)
_, h_t = gru_layer(tokens_embedded)  # h_t = [batch_size, units (k2))
cnn_output = cnn_layer([tokens_embedded, h_t])
model = keras.Model(inputs=[main_input], outputs=cnn_output)
model.compile(optimizer=optimizer,
              loss=loss_func,
              metrics=['accuracy'])
# fit the model
print("Code snippet len: {} label_name len: {}".format(len(code_snippet), len(label_name)))
model.fit(code_snippet, label_name, epochs=50, verbose=0, batch_size=1)
model.summary()


ConvAttention: Tokens shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: C shape = (1, 50, 128), h_t shape = (1, 8)
AttentionFeatures: L_1 shape = (1, 50, 8)
AttentionFeatures: L_2 shape = (1, 50, 8)
AttentionFeatures: L_2 shape  after multiply = (1, 50, 8)
AttentionFeatures: L_feat shape = Tensor("conv_attention_31/attention_features_31/l2_normalize:0", shape=(1, 50, 8), dtype=float32)
ConvAttention: L_feat shape = (1, 50, 8)
AttentionWeights: l_feat shape = (1, 50, 8)
AttentionWeights: attention_weight shape = (1, 50, 1)
Code snippet len: 135 label_name len: 135


ValueError: Invalid argument "batch_size" passed to K.function with TensorFlow backend

In [34]:
model.summary()
# # overfit and evaluate the model 
# loss, accuracy = model.evaluate(code_snippet, label_name, verbose=0)
# print('Accuracy: {}'.format(accuracy * 100))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (1, 50)              0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (1, 50, 128)         40832       main_input[0][0]                 
__________________________________________________________________________________________________
gru_18 (GRU)                    [(1, 8), (1, 8)]     3288        embedding_18[0][0]               
__________________________________________________________________________________________________
conv_attention_18 (ConvAttentio (1, 50, 1)           26529       embedding_18[0][0]               
                                                                 gru_18[0][1]                     
Total para

In [83]:
from data.utils import translate_tokenized_array_to_list_words

translate_tokenized_array_to_list_words(vocab, label_name[4])

['test', 'allocation', 'filtering', '%UNK%', 'shard', 'move']